# Node connectivity fast

This notebook implements and benchmarks a fast algorithm for finding the node connectivity of a graph, defined as the minimum number of nodes that must be removed to make the graph disconnected. It builds on the Esfahanian and Hakimi algorithm used in the NetworkX *node_connectivity* function, leveraging the approximate *local_node_connectivity* function for performance while still returning an exact result.

A full description of the algorithm will be available in the article *Fast and accurate determination of graph node connectivity leveraging approximate methods* by Robert Sinkovits (under review in ACM Journal of Experimental Algorithms).

**Note on timings and results:** All timings were obtained on a MacBook Air (1.8 GHz Intel Core i5, 8 GB 1600 MHz DDR3). For the random graph examples, the graphs and the overall connectivity will vary depending on initial state of the random number generator.

**Note on algorithm nomenclature:** Let $\kappa$ be the exact local node connectivity function and $\kappa_{approx}$ be the approximate local node connectivity function.  
+ **original** algorithm is the Esfahanian and Hakimi algorithm, which exclusively uses $\kappa$
+ **fast** algorithm, introduced here, is a modification of the Esfahanian and Hakimi algorithm to use a combination of $\kappa$ and $\kappa_{approx}$
+ **approximate** algorithm relies entirely on $\kappa_{approx}$

## Imports and functions

Importing packages needed for fast node connectivity

In [1]:
import itertools
from operator import itemgetter
import random
import timeit
import networkx as nx
from networkx.algorithms import approximation as nxap
from networkx.algorithms.connectivity import local_node_connectivity

Importing packages that are normally done within networkx.algorithms.connectivity

In [2]:
from networkx.algorithms.flow import boykov_kolmogorov
from networkx.algorithms.flow import dinitz
from networkx.algorithms.flow import edmonds_karp
from networkx.algorithms.flow import shortest_augmenting_path
from networkx.algorithms.flow import build_residual_network

from networkx.algorithms.connectivity.utils import (build_auxiliary_node_connectivity,
                   build_auxiliary_edge_connectivity)

default_flow_func = edmonds_karp

In [3]:
def node_connectivity_fast(G, s=None, t=None, flow_func=None, startnode=None, 
                           verbose=False, worstcase=False):
    
    '''Fast implementation of the node_connectivity function that leverages
    the approximate local_node_connectivity function. See the NetworkX source 
    code for full explanation of the original node_connectivity function.                                         
                                                                                     
    Parameters                                                                       
    ----------                                                                       
    G : NetworkX graph                                                               
        Undirected graph                                                             
                                                                                     
    s : node                                                                         
        Source node. Optional. Default value: None.                                  
                                                                                     
    t : node                                                                         
        Target node. Optional. Default value: None.                                  
                                                                                     
    flow_func : function                                                             
        A function for computing the maximum flow among a pair of nodes.             
        The function has to accept at least three parameters: a Digraph,             
        a source node, and a target node. And return a residual network              
        that follows NetworkX conventions (see :meth:`maximum_flow` for              
        details). If flow_func is None, the default maximum flow function            
        (:meth:`edmonds_karp`) is used. See below for details. The                   
        choice of the default function may change from version                       
        to version and should not be relied on. Default value: None.   
        
    startnode : node
        Starting node (v) to be used in calculating the connectivity
        Optional. Default value: None
        
    verbose: logical
        If True, print out information that describes progress of function
        Optional. Default value: False
            
    worstcase: logical
        If True, function performs all overhead associated with search for
        optimal starting node (v) and calls to approximate local_node_connectivity
        function, but still forces calls to exact local_node_connectivity function.
        WARNING - you probably don't want to set this to True. Used purely to
        demonstrate that the worst case scenario is not noticeably longer than
        calling the original algorithm
        Optional. Default value: False
                                                                                     
    Returns                                                                          
    -------                                                                          
    K : integer   
    '''

    if (s is not None and t is None) or (s is None and t is not None):
        raise nx.NetworkXError('Both source and target must be specified.')

    # Local node connectivity
    if s is not None and t is not None:
        if s not in G:
            raise nx.NetworkXError('node %s not in graph' % s)
        if t not in G:
            raise nx.NetworkXError('node %s not in graph' % t)
        return local_node_connectivity(G, s, t, flow_func=flow_func)

    # Global node connectivity
    if G.is_directed():
        if not nx.is_weakly_connected(G):
            return 0
        iter_func = itertools.permutations
        # It is necessary to consider both predecessors
        # and successors for directed graphs

        def neighbors(v):
            return itertools.chain.from_iterable([G.predecessors(v),
                                                  G.successors(v)])
    else:
        if not nx.is_connected(G):
            return 0
        iter_func = itertools.combinations
        neighbors = G.neighbors

    # Reuse the auxiliary digraph and the residual network
    H = build_auxiliary_node_connectivity(G)
    R = build_residual_network(H, 'capacity')
    kwargs = dict(flow_func=flow_func, auxiliary=H, residual=R)

    # If the startnode was specified, use this as starting point (v)
    # for the node connectivity calculations. Otherwise, choose the
    # "best" option from among the nodes of lowest degree, where the
    # best node is the one where the approximation to the local node
    # connectivity is greater than or equal to the minimum node degree
    # with the highest frequency
    
    if startnode:
        v = startnode
        K = G.degree[startnode]
    else:
        # Set tunable parameters for selecting best node
        max_candidates = 10
        max_targets = 100
        
        # Pick a node with minimum degree
        # Node connectivity is bounded by degree.
        v, K = min(G.degree(), key=itemgetter(1))
        nexact_best = max_targets
        
        # Now find all nodes of that degree
        candidates = []
        for node,d in G.degree():
            if d == K:
                candidates.append(node)
            
        # Restrict ourselves to a subset of these nodes
        candidates = candidates[0:min(max_candidates,len(candidates))]
        
        # Now test number of times that the approximate local connectivity between
        # each candidate and a random subset of the non-adjacent nodes in the graph
        # is less than the minimum node degee
        if verbose:
            print('Searching for best starting node')
            print('node   nexact')
        
        for s in candidates:
            nexact = 0
            targets = set(G) - set(neighbors(s)) - set([s])
            targets = random.sample(targets, min(max_targets, len(targets)))
            for t in targets:
                if nxap.local_node_connectivity(G, s, t) < K:
                    nexact += 1
            if verbose:
                print(s, nexact)
            if nexact < nexact_best:
                nexact_best = nexact
                v = s
        
    # Keep track the number of calls to approximate and exact
    # local node connectivity functions
    napprox = 0
    nexact  = 0
    
    # This is a hack to ensure that we accumulate all the overhead associated
    # with finding a good choice for the starting node AND actually use the same
    # starting node as in the original algorithm. Allows for a more accurate
    # performace comparison since the expense of evaluating the exact local
    # node connectivity functions depends on choice of source
    if worstcase:
        v, K = min(G.degree(), key=itemgetter(1))
    
    # compute local node connectivity with all its non-neighbors nodes
    for w in set(G) - set(neighbors(v)) - set([v]):
        kwargs['cutoff'] = K
        napprox += 1
        if nxap.local_node_connectivity(G, v, w) < K or worstcase:
            nexact += 1
            K = min(K, local_node_connectivity(G, v, w, **kwargs))
    # Also for non adjacent pairs of neighbors of v
    for x, y in iter_func(neighbors(v), 2):
        if y in G[x]:
            continue
        kwargs['cutoff'] = K
        napprox += 1
        if nxap.local_node_connectivity(G, x, y) < K or worstcase:
            nexact += 1
            K = min(K, local_node_connectivity(G, x, y, **kwargs))
            
    if verbose:
        print('node', v, 'napprox = ', napprox, 'nexact = ', nexact)

    return K

## Benchmarks

The following benchmarks illustrate how the performance of the fast algorithm compares to that of the original Esfahanian and Hakimi algorithm as deployed in NetworkX. The first sets of benchmarks are for graphs of various sizes and edge densities generated using standard random models (Barabasi-Albert, Erdos-Renyi, Watts-Strogatz). 

This is followed by benchmarks for the largest embedded 3-component through 7-component extracted from a large anonymized social network. We also include some intermediate graphs that were obtained during the iterative process to identify the *K*-components. For example, *social_largest_pre6component.net* is the last 5-connected graph we obtained before reducing to a 6-component.

### Barabasi-Albert graph benchmarks

The following example compares the performance of the fast and original node connectivity algorithms for random graphs of various sizes and densities generated by the **Barabasi-Albert** model. For the collection of graphs n = {1000, 2000, 4000, 8000, 16000} and m = {4, 6, 8, 10} the total run time is approximately **8 hours**. This can reduced significantly by omitting the larger values of n and m.

In [7]:
print("Barabasi-Albert graph benchmarks\n")
seedbase = 1456789356

for n in [1000, 2000, 4000, 8000, 16000]:
    for m in [4, 6, 8, 10]:
        G = nx.barabasi_albert_graph(n, m, seed=seedbase+n+m)
        print('n = ', n, 'm = ',m, '#nodes = ', len(G.nodes()), '#edges = ', len(G.edges()))
        
        start_time = timeit.default_timer()
        K = node_connectivity_fast(G)
        print('Fast algorithm        K:', K, 't:', timeit.default_timer() - start_time)
        
        start_time = timeit.default_timer()
        K = nx.node_connectivity(G)
        print('Original algorithm    K:', K, 't:', timeit.default_timer() - start_time)
        
        start_time = timeit.default_timer()
        K = nxap.node_connectivity(G)
        print('Approximate algorithm K:', K, 't:', timeit.default_timer() - start_time)
        print()

Barabasi-Albert graph benchmarks

n =  1000 m =  4 #nodes =  1000 #edges =  3984
Fast algorithm        K: 4 t: 0.3638930040001469
Original algorithm    K: 4 t: 15.310807535999857
Approximate algorithm K: 4 t: 0.1020567509999637

n =  1000 m =  6 #nodes =  1000 #edges =  5964
Fast algorithm        K: 6 t: 0.4573221040000135
Original algorithm    K: 6 t: 21.066792194999834
Approximate algorithm K: 6 t: 0.1707212610001534

n =  1000 m =  8 #nodes =  1000 #edges =  7936
Fast algorithm        K: 8 t: 0.7526207020000584
Original algorithm    K: 8 t: 28.07024301199999
Approximate algorithm K: 8 t: 0.2438413650002076

n =  1000 m =  10 #nodes =  1000 #edges =  9900
Fast algorithm        K: 10 t: 1.3008893159999388
Original algorithm    K: 10 t: 31.144300595999994
Approximate algorithm K: 10 t: 0.32582205800008524

n =  2000 m =  4 #nodes =  2000 #edges =  7984
Fast algorithm        K: 4 t: 0.7578165339996303
Original algorithm    K: 4 t: 52.31251421800016
Approximate algorithm K: 4 t: 0.317939

### Erdos-Renyi graph benchmarks

The following example compares the performance of the fast and original node connectivity algorithms for random graphs of various sizes and densities generated by the **Erdos-Renyi** model. For the collection of graphs n = {1000, 2000, 4000} and p = {0.01, 0.02, 0.03} the total run time is approximately **5 hours**. This can reduced significantly by omitting the larger values of n and p.

In [13]:
print("Erdos-Renyi graph benchmarks\n")
seedbase = 1456789356

for n in [1000, 2000, 4000]:
    for p in [0.01, 0.02, 0.03]:
        G = nx.erdos_renyi_graph(n, p, seed=int(seedbase+n+p*1000))
        print('n = ', n, 'p = ', p, '#nodes = ', len(G.nodes()), '#edges = ', len(G.edges()))
        
        start_time = timeit.default_timer()
        K = node_connectivity_fast(G)
        print('Fast algorithm        K:', K, 't:', timeit.default_timer() - start_time)
        
        start_time = timeit.default_timer()
        K = nx.node_connectivity(G)
        print('Original algorithm    K:', K, 't:', timeit.default_timer() - start_time)
        
        start_time = timeit.default_timer()
        K = nxap.node_connectivity(G)
        print('Approximate algorithm K:', K, 't:', timeit.default_timer() - start_time)
        print()

Erdos-Renyi graph benchmarks

n =  1000 p =  0.01 #nodes =  1000 #edges =  4965
Fast algorithm        K: 3 t: 0.3223687880017678
Original algorithm    K: 3 t: 16.86385721500119
Approximate algorithm K: 3 t: 0.10402909399999771

n =  1000 p =  0.02 #nodes =  1000 #edges =  9988
Fast algorithm        K: 8 t: 0.5287282170029357
Original algorithm    K: 8 t: 34.84225387499464
Approximate algorithm K: 8 t: 0.3121367839994491

n =  1000 p =  0.03 #nodes =  1000 #edges =  15004
Fast algorithm        K: 13 t: 1.2462669480009936
Original algorithm    K: 13 t: 58.35360807899997
Approximate algorithm K: 13 t: 0.5911985839993577

n =  2000 p =  0.01 #nodes =  2000 #edges =  20019
Fast algorithm        K: 6 t: 1.30561148400011
Original algorithm    K: 6 t: 113.60037790099886
Approximate algorithm K: 6 t: 0.6181156500024372

n =  2000 p =  0.02 #nodes =  2000 #edges =  40007
Fast algorithm        K: 21 t: 5.558848368003964
Original algorithm    K: 21 t: 311.78672886899585
Approximate algorithm K: 21

### Watts-Strogatz graph benchmarks

The following example compares the performance of the fast and original node connectivity algorithms for random graphs of various sizes and densities generated by the **Watts-Strogatz** model. For the collection of graphs n = {1000, 2000, 4000, 8000, 16000} and k = {5, 7, 9} the total run time is approximately **3 hours**. This can reduced significantly by omitting the larger values of n.

In [10]:
print("Watts-Strogatz graph benchmarks (p=0.10)\n")
seedbase = 1456789356

for n in [1000, 2000, 4000, 8000, 16000]:
    for k in [5, 7, 9]:
        G = nx.connected_watts_strogatz_graph(n, k, 0.10, seed=seedbase+n+k)
        print('n = ', n, 'k = ', k, '#nodes = ', len(G.nodes()), '#edges = ', len(G.edges()))
        
        start_time = timeit.default_timer()
        K = node_connectivity_fast(G)
        print('Fast algorithm        K:', K, 't:', timeit.default_timer() - start_time)
        
        start_time = timeit.default_timer()
        K = nx.node_connectivity(G)
        print('Original algorithm    K:', K, 't:', timeit.default_timer() - start_time)
        
        start_time = timeit.default_timer()
        K = nxap.node_connectivity(G)
        print('Approximate algorithm K:', K, 't:', timeit.default_timer() - start_time)
        print()

Watts-Strogatz graph benchmarks (p=0.10)

n =  1000 k =  5 #nodes =  1000 #edges =  2000
Fast algorithm        K: 2 t: 0.3878485870009172
Original algorithm    K: 2 t: 11.00787210999988
Approximate algorithm K: 2 t: 0.23016156400262844

n =  1000 k =  7 #nodes =  1000 #edges =  3000
Fast algorithm        K: 3 t: 0.4226154670031974
Original algorithm    K: 3 t: 13.453259275993332
Approximate algorithm K: 3 t: 0.3545397919951938

n =  1000 k =  9 #nodes =  1000 #edges =  4000
Fast algorithm        K: 5 t: 0.7518266180049977
Original algorithm    K: 5 t: 18.72791978299938
Approximate algorithm K: 5 t: 0.5083429989972501

n =  2000 k =  5 #nodes =  2000 #edges =  4000
Fast algorithm        K: 2 t: 1.0354474229970947
Original algorithm    K: 2 t: 38.352676697999414
Approximate algorithm K: 2 t: 0.5466014909980004

n =  2000 k =  7 #nodes =  2000 #edges =  6000
Fast algorithm        K: 3 t: 0.8417195020010695
Original algorithm    K: 3 t: 50.69690547600476
Approximate algorithm K: 3 t: 0.819

### Social network benchmarks

The following example compares the fast and original node connectivity algorithms on large embedded *K*-components extracted from an anonymized social network. The benchmark for the first five graphs should take about **68 minutes**. Including the 6th, 7th and 8th graphs increases the run time by about **45 minutes**, **3 hours** and **8 hours**, respectively. 

In [7]:
graphs = ['social_network/social_largest_7component.net',
'social_network/social_largest_pre7component.net',
'social_network/social_largest_6component.net',
'social_network/social_largest_pre6component.net',
'social_network/social_largest_5component.net']

graphs.append('social_network/social_largest_pre5component.net') # Adds 45 minutes
graphs.append('social_network/social_largest_4component.net')    # Adds 3 hours
graphs.append('social_network/social_largest_3component.net')    # Adds 8 hours

In [8]:
print("Social network benchmarks\n")

for graph in graphs:
    G = nx.read_pajek(graph)
    
    print('graph ', graph)
    print('#nodes = ', len(G.nodes()), '#edges = ', len(G.edges()))
    
    start_time = timeit.default_timer()
    K = node_connectivity_fast(G)
    print('Fast algorithm        K:', K, 't:', timeit.default_timer() - start_time)
        
    start_time = timeit.default_timer()
    K = nx.node_connectivity(G)
    print('Original algorithm    K:', K, 't:', timeit.default_timer() - start_time)
        
    start_time = timeit.default_timer()
    K = nxap.node_connectivity(G)
    print('Approximate algorithm K:', K, 't:', timeit.default_timer() - start_time)
    print()

Social network benchmarks

graph  social_network/social_largest_7component.net
#nodes =  542 #edges =  3704
Fast algorithm        K: 7 t: 2.7488822170125786
Original algorithm    K: 7 t: 14.200604388985084
Approximate algorithm K: 5 t: 0.3714499000052456

graph  social_network/social_largest_pre7component.net
#nodes =  573 #edges =  3889
Fast algorithm        K: 6 t: 1.0175660379754845
Original algorithm    K: 6 t: 15.818272449978394
Approximate algorithm K: 6 t: 0.771714211005019

graph  social_network/social_largest_6component.net
#nodes =  3089 #edges =  19002
Fast algorithm        K: 6 t: 69.58663278899621
Original algorithm    K: 6 t: 468.2864766189887
Approximate algorithm K: 4 t: 13.969473876000848

graph  social_network/social_largest_pre6component.net
#nodes =  3636 #edges =  22166
Fast algorithm        K: 5 t: 45.50614229097846
Original algorithm    K: 5 t: 520.5763859889994
Approximate algorithm K: 3 t: 12.500374520983314

graph  social_network/social_largest_5component.net


## Supplementary examples

In this section, we explore several points that were discussed in the manuscript

* Impact that choice of the starting node can have on the performance of the fast algorithm

* Worst case performance scenarios

### Effect of starting node choice on performance

The following example illustrates how the run time of the fast node connectivity algorithm can depend strongly on the choice of the selected node that is used as the source for most of the local node connectivity calculations ('v' in the Esfahanian and Hakimi algorithm).

Using the first 15 nodes, this should take about **27 minutes**, but see comments below to extend to more calculations. The counters *napprox* and *nexact* report the number of times that the approximate and exact local node connectivity algorithms are called in the process of finding the overall node connectivity of the graph.

In [14]:
print("Effect of starting node choice\n")

G = nx.read_pajek('social_network/social_largest_6component.net')
candidates = []
v, K = min(G.degree(), key=itemgetter(1))
for v,d in G.degree():
    if d == K:
        candidates.append(v)

# Comment out following line if you want to test up to
# all 451 nodes of minimum degree. This will take about 15 hours.

# candidates = candidates[0:min(15,len(candidates))]
      
for node in candidates:
    start_time = timeit.default_timer()
    K = node_connectivity_fast(G, startnode=node, verbose=True)
    print('K:', K, 't:', timeit.default_timer() - start_time)

Effect of starting node choice

node a6457 napprox =  3085 nexact =  245
K: 6 t: 71.33846620999975
node c481 napprox =  3083 nexact =  270
K: 6 t: 89.13355598400085
node f205 napprox =  3087 nexact =  321
K: 6 t: 98.7356709079977
node d1225 napprox =  3086 nexact =  320
K: 6 t: 71.48745518000214
node d3810 napprox =  3090 nexact =  258
K: 6 t: 71.81945162000193
node d7988 napprox =  3088 nexact =  663
K: 6 t: 121.95665797699621
node h1998 napprox =  3087 nexact =  267
K: 6 t: 65.28719663499942
node h6722 napprox =  3083 nexact =  249
K: 6 t: 64.63903141499759
node h8264 napprox =  3089 nexact =  249
K: 6 t: 69.65020606100006
node b0079 napprox =  3084 nexact =  224
K: 6 t: 62.589603791995614
node b6573 napprox =  3087 nexact =  1802
K: 6 t: 303.12492093000037
node b6862 napprox =  3082 nexact =  2211
K: 6 t: 358.1430213450003
node b9947 napprox =  3087 nexact =  263
K: 6 t: 64.98657666300278
node i2243 napprox =  3088 nexact =  288
K: 6 t: 63.0135247510043
node e1942 napprox =  3087 ne

### Worst case performance scenario

In the worst case performance scenario, the approximate local node connectivity function always returns a value that is less than the running value for *K*. This results in additional overhead since the exact *and* approximate functions need to be evaluated for each node pair. We show here that the even in the highly unlikely event that this occurs, the impact is minimal and typically less than the run-to-run variation.

Setting worstcase to True in the calls to *node_connectivity_fast* below results in the evaluation of the approximate local node connectivity algorithm, but with the answer ignored so that the approximate local node connectivity algorithm still has to be called. Also, to allow for a better comparison, it enforces that the same starting node (v) will be used in both the fast and original algorithms, while accounting for all overhead due to the search for an optimal starting node.

Each of the benchmarks below take about **10-12 minutes** to complete: five Barabasi-Albert graphs with n=2000, m=4; five Erdos-Renyi graphs with n=1000, p=0.03; five Erdos-Renyi graphs with n=1000, p=0.03; five Watts-Strogatz graphs with n=2000, k=9, p=0.10

In [10]:
print("Worst case Barabasi-Albert graphs\n")
for i in range(5):
    G = nx.barabasi_albert_graph(2000, 4)
    start_time = timeit.default_timer()
    K = node_connectivity_fast(G, worstcase=True)
    print('Fast worst case    K:', K, 't:', timeit.default_timer() - start_time)       
    start_time = timeit.default_timer()
    K = nx.node_connectivity(G)
    print('Original algorithm K:', K, 't:', timeit.default_timer() - start_time)
    print()

Worst case Barabasi-Albert graphs

Fast worst case    K: 4 t: 67.24951901199529
Original algorithm K: 4 t: 68.09812424800475

Fast worst case    K: 4 t: 67.47647145600058
Original algorithm K: 4 t: 66.44104401301593

Fast worst case    K: 4 t: 67.6486507189984
Original algorithm K: 4 t: 67.18817964597838

Fast worst case    K: 4 t: 68.08023200300522
Original algorithm K: 4 t: 66.77867178898305

Fast worst case    K: 4 t: 68.81827678499394
Original algorithm K: 4 t: 67.95580610699835



In [11]:
print("Worst case Erdos-Renyi graphs\n")
for i in range(5):
    G = nx.erdos_renyi_graph(1000, 0.03)
    start_time = timeit.default_timer()
    K = node_connectivity_fast(G, worstcase=True)
    print('Fast worst case    K:', K, 't:', timeit.default_timer() - start_time)       
    start_time = timeit.default_timer()
    K = nx.node_connectivity(G)
    print('Original algorithm K:', K, 't:', timeit.default_timer() - start_time)
    print()

Worst case Erdos-Renyi graphs

Fast worst case    K: 13 t: 57.87264456198318
Original algorithm K: 13 t: 57.452433472004486

Fast worst case    K: 15 t: 62.66547342800186
Original algorithm K: 15 t: 60.96831700799521

Fast worst case    K: 12 t: 55.811364414024865
Original algorithm K: 12 t: 55.2875133670168

Fast worst case    K: 14 t: 59.820123565004906
Original algorithm K: 14 t: 58.59856275099446

Fast worst case    K: 15 t: 61.24938508900232
Original algorithm K: 15 t: 61.64648888402735



In [12]:
print("Worst case Watts-Strogatz graphs\n")
for i in range(5):
    G = nx.connected_watts_strogatz_graph(2000, 9, 0.10)
    start_time = timeit.default_timer()
    K = node_connectivity_fast(G, worstcase=True)
    print('Fast worst case    K:', K, 't:', timeit.default_timer() - start_time)       
    start_time = timeit.default_timer()
    K = nx.node_connectivity(G)
    print('Original algorithm K:', K, 't:', timeit.default_timer() - start_time)
    print()

Worst case Watts-Strogatz graphs

Fast worst case    K: 5 t: 72.55485465200036
Original algorithm K: 5 t: 71.14637235199916

Fast worst case    K: 5 t: 71.1597859460162
Original algorithm K: 5 t: 69.51093091000803

Fast worst case    K: 5 t: 74.10120390402153
Original algorithm K: 5 t: 71.23631437600125

Fast worst case    K: 5 t: 71.54087027499918
Original algorithm K: 5 t: 69.03208708800958

Fast worst case    K: 5 t: 73.00514114499674
Original algorithm K: 5 t: 71.51044943201123

